# Forced initial state for a droplet in 2D

In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#20.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("ForcedDropletDeformation2D");

Project name is set to 'ForcedDropletDeformation2D'.
Opening existing database 'D:\local\ForcedDropletDeformation2D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
var myBatch = ExecutionQueues[0];
//var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ]


## Grid Creation

In [ ]:
Dictionary<string, IGridInfo[]> gridTypes = new Dictionary<string, IGridInfo[]>();

In [ ]:
int[] Resolutions = new int[] { 12 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
double scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"ForcedDroplet2D_{Res}x{2*Res}";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid2D.Cartesian2DGrid(xNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("wallBC", Grids);

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\ForcedDropletDeformation2D'.
Grid already found in database - identifid by name ForcedDroplet2D_12x24


In [ ]:
int[] Resolutions = new int[] { 12 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
double scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"ForcedDroplet2D_{Res}x{2*Res}";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid2D.Cartesian2DGrid(xNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Pressure_Outlet.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("pressureOutletBC", Grids);

Grid already found in database - identifid by name ForcedDroplet2D_12x24


## Prescribed level-set evolution (Spherical harmonics)

We define the initial deformation by the inviscid Rayleigh frequency $\omega_m^2 = \frac{\sigma}{\rho R_0^3}(m(m-1)(m+2))$ 

In [ ]:
var Phi1Init = new Formula(
"Phi1",
true,
"using ilPSP.Utils; " + 
"double Phi1(double[] X, double t) { " + 
"     " + 
"    double theta = Math.Atan2(X[0],X[1]); " + 
"    double omega = Math.Sqrt(8); " + 
"    double R =    0.966781*SphericalHarmonics.MyRealSpherical(0, 0, theta, 0.0) " + 
"                +      0.4*Math.Sin(omega*t)*SphericalHarmonics.MyRealSpherical(2, 0, theta, 0.0); " + 
"    return X.L2Norm() - R; " + 
"}");

In [ ]:
IBoundaryAndInitialData[] Phi_iCase = new IBoundaryAndInitialData[]  { Phi1Init };

## Setup of control objects for all solver runs

The end time is set at $t_{end} = \frac{\pi}{2\omega}$

In [ ]:
double[] tend = new double[] { 0.555, 0.288, 0.185, 0.555, 0.185};

In [ ]:
string[] gridTypeKeys = new string[] { "wallBC", "pressureOutletBC" };
(int Case, double Ohnesorg)[] Cases = new[] { (1, 0.1) }; // { (1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.56) };
(double dt, int timesteps)[] Cases_time = new[] { (5e-3, 111), (2e-3, 144), (2e-3, 93), (5e-3, 111), (5e-3, 37)};
int[] AMRlevels = new int[] {1, 2, 2, 1, 1};

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
bool[] useNewton = new bool[] { false };

string grdKey = gridTypeKeys[1];
Grids = gridTypes[grdKey];

foreach(bool bNewton in useNewton) {
foreach(int k in DegreeS) {
foreach(var grd in Grids) {
foreach(var myCase in Cases) {
    long J = grd.NumberOfCells;
    int AMRlvl = AMRlevels[myCase.Case-1];
    string JobName = $"FD2DInit_J{J}k{k}_{grdKey}_amr{AMRlvl}_case{myCase.Case}";
    if(bNewton) {
        JobName = JobName + "_Newton";
    }
    Console.WriteLine("Case: " + JobName);

    var C = new XNSE_Control();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    
    C.InitialValues.Add("Phi", Phi_iCase[myCase.Case - 1]);
    C.InitialValues.Add("Pressure#A", new Formula("(X) => 1.0", false));
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = myCase.Ohnesorg/1000;
    C.PhysicalParameters.mu_B = myCase.Ohnesorg/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.Prescribed;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
    }
    C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = Cases_time[myCase.Case - 1].dt;
    C.NoOfTimesteps = Cases_time[myCase.Case - 1].timesteps;
    
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    //C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    //C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    //C.PostprocessingModules.Add(new EnergyLogging());
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}
}
}

Case: FD2DInit_J288k3_pressureOutletBC_amr0_case1


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal");
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal");
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,FD2DInit_J288k3_pressureOutletBC_amr0_case1


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Deploying job FD2DInit_J288k3_pressureOutletBC_amr0_case1 ... 
Deploying executables and additional files ...
Deployment directory: D:\local\binaries\ForcedDropletDeformation2D-XNSE_Solver2022Mar16_131356
copied 50 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.



In [ ]:
wmg.AllJobs

#0: FD2DInit_J288k3_wallBC_amr0_case1: FailedOrCanceled (MiniBatchProcessor client  LocalPC @D:\local\binaries)	FD2DInit_J288k3_wallBC_amr0_case1: FailedOrCanceled (MiniBatchProcessor client  LocalPC @D:\local\binaries)
#1: FD2DInit_J288k3_pressureOutletBC_amr0_case1: PendingInExecutionQueue (MiniBatchProcessor client  LocalPC @D:\local\binaries)	FD2DInit_J288k3_pressureOutletBC_amr0_case1: PendingInExecutionQueue (MiniBatchProcessor client  LocalPC @D:\local\binaries)
